<a href="https://colab.research.google.com/github/Glaudemias/Projeto_COVID_Sertao_CENTRAL/blob/main/Notebooks/PROJETO_PARTE_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROJETO MÓDULO 3 - BOOTCAMP DATASCIENCE APLICADO 2 - PARTE 1**
---
## Análise e Previsão de Séries Temporais dos dados de Covid-19 no Sertão Central, CE

Nesse Notebook será realizado a limpeza e tratamento da base de dados obtida pelo portal Corona Virús Brasil, você pode acessa-lo clicando [aqui:](https://covid.saude.gov.br/)

# **IMPORTAÇÃO DAS BIBLIOTECAS**

Bibliotecas Básicas

In [ ]:
#Biblioteca de tratamento de dados
import pandas as pd

#Biblioteca que retira os avisos de erro
import warnings
warnings.filterwarnings('ignore')

Instalando o Geopandas!

In [ ]:
#Instalação
!!pip install geopandas

#Biblioteca responsável pela visualização de dados espaciais do tipo shapefile
import geopandas as gpd
from geopandas import GeoDataFrame

Biblioteca de exportação de arquivos tratados

In [ ]:
from google.colab import files

**Criando Funções**

In [ ]:
def filtraestado(dataframe, coluna,estado):
  dataframe = dataframe[dataframe[coluna]==estado]
  return dataframe

In [ ]:
def excluiNA (dataframe,coluna1,coluna2):
  filtro = (dataframe[coluna1].notna()) & (dataframe[coluna2].notna())
  dataframe = dataframe[filtro]
  return dataframe

In [ ]:
def prepara_mapa(dataframe,coluna):
  codigos = []
  for cod in dataframe[coluna]:
    codigos.append(cod[:-1])
  dataframe['codmun'] = codigos
  dataframe['id'] = dataframe['ID']
  dataframe['cod_IBGE'] = dataframe['CD_GEOCODM']
  dataframe = dataframe.drop(columns = ['ID','CD_GEOCODM','NM_MUNICIP'])
  dataframe['codmun'] = dataframe['codmun'].astype(int)
  return dataframe


**Listas de Regiões**

In [ ]:
#Definindo as regiões cearenses

cariri = ['Abaiara','Altaneira','Antonina do Norte', 'Araripe', 'Assaré','Aurora','Barbalha','Barro', 
          'Brejo Santo', 'Campos Sales', 'Caririaçu', 'Crato', 'Farias Brito','Granjeiro', 'Jardim', 'Jati','Juazeiro do Norte', 'Lavras da Mangabeira', 
           'Mauriti', 'Milagres', 'Missão Velha', 'Nova Olinda', 'Penaforte', 'Porteiras', 'Potengi', 'Salitre', 'Santana do Cariri','Tarrafas','Várzea Alegre']

centro_sul=['Acopiara', 'Baixio', 'Cariús', 'Catarina', 'Cedro', 'Icó', 'Iguatu','Ipaumirim', 'Jucás', 'Orós', 'Quixelô', 'Saboeiro','Umari']

reg_metro = ['Aquiraz', 'Caucaia', 'Cascavel', 'Chorozinho', 'Eusébio','Fortaleza', 'Guaiúba', 'Horizonte', 'Itaitinga', 'Maracanaú', 'Maranguape', 'Pacajus', 'Pacatuba',
           'Paracuru', 'Paraipaba', 'Pindoretama', 'São Luís do Curu', 'São Gonçalo do Amarante', 'Trairi']

litoral_leste=['Aracati', 'Beberibe', 'Fortim', 'Icapuí', 'Itaiçaba','Jaguaruana']

litoral_norte =['Acaraú', 'Barroquinha', 'Bela Cruz', 'Camocim', 'Chaval', 'Cruz','Granja', 'Itarema', 'Jijoca de Jericoacoara', 'Marco', 'Martinópole', 'Morrinhos','Uruoca']

litoral_oeste = ['Amontada', 'Apuiarés', 'General Sampaio','Irauçuba','Itapajé','Itapipoca','Miraíma', 'Pentecoste', 'Tejuçuoca', 'Tururu', 'Umirim','Uruburetama']

maciço_baturite = ['Acarape','Aracoiaba','Aratuba', 'Barreira', 'Baturité', 'Capistrano','Guaramiranga', 'Itapiúna', 'Mulungu', 'Ocara', 'Pacoti','Palmácia', 'Redenção']

serra_da_ibiapaba = ['Carnaubal', 'Croatá','Guaraciaba do Norte','Ibiapina','Ipu','São Benedito','Tianguá','Ubajara','Viçosa do Ceará']

sertao_central = ['Banabuiú', 'Choró','Deputado Irapuan Pinheiro','Ibaretama','Ibicuitinga','Milhã','Mombaça', 
           'Pedra Branca','Piquet Carneiro','Quixadá', 'Quixeramobim','Senador Pompeu','Solonópole']

sertao_canide=['Boa Viagem', 'Canindé', 'Caridade', 'Itatira', 'Madalena', 'Paramoti']

sertao_sobral=['Alcântaras', 'Cariré','Coreaú', 'Forquilha','Frecheirinha','Graça','Groaíras','Massapê','Meruoca','Moraújo','Mucambo','Pacujá','Pires Ferreira',
          'Reriutaba','Santana do Acaraú','Senador Sá','Sobral','Varjota']

sertao_crateus =['Ararendá','Catunda','Crateús','Hidrolândia','Independência','Ipaporanga','Ipueiras','Monsenhor Tabosa','Nova Russas','Novo Oriente', 
           'Poranga','Santa Quitéria','Tamboril']

sertao_inhamu = ['Aiuaba','Arneiroz','Parambu','Quiterianópolis','Tauá']

vale_do_jaguaribe=['Alto Santo','Ererê','Iracema','Jaguaretama','Jaguaribara','Jaguaribe','Limoeiro do Norte','Morada Nova','Palhano', 
           'Pereiro','Potiretama','Quixeré','Russas','São João do Jaguaribe','Tabuleiro do Norte', 'Ererê','Ereré']


In [ ]:
#função de regiao
def regiao(item):
  if item in cariri:
    return 'Cariri'
  if item in centro_sul:
    return 'Centro-Sul'
  if item in reg_metro:
    return 'Região Metropolitana'
  if item in litoral_leste:
    return 'Litoral Leste'
  if item in litoral_norte:
    return 'Litoral Norte'
  if item in litoral_oeste:
    return 'Litoral Oeste'
  if item in maciço_baturite:
    return 'Maciço de Baturité'
  if item in serra_da_ibiapaba:
    return 'Serra da Ibiapaba'
  if item in sertao_central:
    return 'Sertão Central'
  if item in sertao_canide:
    return 'Sertão de Canidé'
  if item in sertao_sobral:
    return 'Sertão de Sobral'
  if item in sertao_crateus:
    return 'Sertao dos Crateús'
  if item in sertao_inhamu:
    return 'Sertão dos Inhamus'
  if item in vale_do_jaguaribe:
    return 'Vale do Jaguaribe'
  else:
    return 'não identificado'

# **TRATANDO OS DATASETS**

A Base de dados assim que baixada do site oficial, vem comprimida num arquivo zip e é dividida em 3 partes.

Eles possuem as seguintes informações:

* **região:** Contém as regiões do país e ainda os números somados que são relativos ao país, seus valores são: *Brasil, Norte, Nordeste, Sul, Sudeste e Centro-Oeste*. 

* **estado:** Contém as Unidades Federativas do país, possuindo valores NAN quando a linha for relativa a Região classificada como Brasil, seus valores são as 27 unidades da federação.

* **municipio:** Contém o nome de todos os municipios do país e possui valor NAN quando a linha for relativa classificada como Brasil.

* **coduf:** Código da Unidade federativa, esses códigos são utilizados na emissão de Documentos Fiscais Eletrônicos (NFC-e, NF-e, CT-e, MDF-e e CF-e).

* **codmun:** Código dos municipios referente aos valores de identificação do IBGE.

>> **Campos de Região de Saúde**
As regiões de Saúde são consideradas um espaço geográfico contínuo constituído por agrupamentos de municípios limítrofes, delimitado a partir de identidades culturais, econômicas e sociais e de redes de comunicação e infraestrutura de transportes compartilhados, com a finalidade de integrar a organização, o planejamento e a execução de ações e serviços de saúde.

* **codRegiaoSaude:**  Código de identificação de cada Região de Saúde

* **nomeRegiaoSaude:** Nome de cada Região de Saúde.

* **data:** Data relativa ao caso novo, notificado.

* **semanaEpi:** Semana Epidemológica referente a notificação

* **populacaoTCU2019:** Indica a população estimada de 2019 utilizada pelo TCU de onde foi registrada a observação.

* **casosAcumulado:** Indica o total de casos acumulados em uma data especifica.

* **casosNovos:** Número de casos novos notificados em uma data especifica

* **obitosAcumulado:** Indica o total de obitos acumulados em uma data especifica.

* **obitosNovos:** Número de obitos novos notificados em uma data especifica

* **Recuperadosnovos:** Número de obitos novos notificados em uma data especifica

* **emAcompanhamentoNovos:** Número de casos em acompanhamento notificados em uma data especifica

* **interior/metropolitana:** Indica onde a observação foi registrada na região do estado, se no interior (valor 0) ou na região metropolitana (valor 1)


## DATAFRAME CASOS CE

In [ ]:
#Parte 1
dados_brutos1=pd.read_csv('/content/drive/MyDrive/BOOTCAMP - ALURA/Projeto/Arquivos Projeto 3/HIST_PAINEL_COVIDBR_2020_Parte1_25jun2021.csv',sep=';')
dados_brutos1.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,1,0,0,NaN,NaN,NaN


In [ ]:
#Parte 2
dados_brutos2=pd.read_csv('/content/drive/MyDrive/BOOTCAMP - ALURA/Projeto/Arquivos Projeto 3/HIST_PAINEL_COVIDBR_2020_Parte2_25jun2021.csv',sep=';')
dados_brutos2.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-08-01,31,210147125.0,2707877,45392,93563,1088,1865729.0,748585.0,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-08-02,32,210147125.0,2733677,25800,94104,541,1883677.0,755896.0,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-08-03,32,210147125.0,2750318,16641,94665,561,1912319.0,743334.0,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-08-04,32,210147125.0,2801921,51603,95819,1154,1970767.0,735335.0,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-08-05,32,210147125.0,2857597,55676,97240,1421,2020637.0,741180.0,NaN


In [ ]:
#Parte 3
dados_brutos3=pd.read_csv('/content/drive/MyDrive/BOOTCAMP - ALURA/Projeto/Arquivos Projeto 3/HIST_PAINEL_COVIDBR_Parte3_25jun2021.csv',sep=';')
dados_brutos3.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-01-01,53,210147125.0,7700578.0,24605,195411,462,6756284.0,748883.0,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-01-02,53,210147125.0,7716405.0,15827,195725,314,6769420.0,751260.0,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-01-03,1,210147125.0,7733746.0,17341,196018,293,6813008.0,724720.0,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-01-04,1,210147125.0,7753752.0,20006,196561,543,6875230.0,681961.0,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-01-05,1,210147125.0,7810400.0,56648,197732,1171,6963407.0,649261.0,NaN


Para esse projeto só sera analisado os municipios relacionados ao estado do CEARÁ, portanto todos os dataframes serão filtrados pelo estado *CE*, atráves da função **filtraestado**

In [ ]:
#Filtrando os dados pelo estado do Ceará
dados1 = filtraestado(dados_brutos1,'estado','CE')
dados2 = filtraestado(dados_brutos2,'estado','CE')
dados3 = filtraestado(dados_brutos3,'estado','CE')

Feito isso o próximo passo é concatenar os dataframes em uma única base de dados, chamada **dados_CE**

In [ ]:
#concatenando os dataframes
dados_CE = pd.concat([dados1, dados2, dados3])
dados_CE.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
1580,Nordeste,CE,NaN,23,NaN,NaN,NaN,2020-02-25,9,9132078.0,0.0,0,0,0,NaN,NaN,NaN
1581,Nordeste,CE,NaN,23,NaN,NaN,NaN,2020-02-26,9,9132078.0,0.0,0,0,0,NaN,NaN,NaN
1582,Nordeste,CE,NaN,23,NaN,NaN,NaN,2020-02-27,9,9132078.0,0.0,0,0,0,NaN,NaN,NaN
1583,Nordeste,CE,NaN,23,NaN,NaN,NaN,2020-02-28,9,9132078.0,0.0,0,0,0,NaN,NaN,NaN
1584,Nordeste,CE,NaN,23,NaN,NaN,NaN,2020-02-29,9,9132078.0,0.0,0,0,0,NaN,NaN,NaN


Observando os dados dos dataframes é possivel observar que quando a coluna `codmun`ou `municipio`são iguais a *NaN*, é porque esses dados são referentes aos valores totais de cada estado. Nesse caso deveremos fazer uma seleção dos dados onde estes não são iguais a *NaN*

In [ ]:
display(dados_CE['municipio'].isnull().sum().sum())
display(dados_CE['codmun'].isnull().sum().sum())

943

487

In [ ]:
display(dados_CE['municipio'].isna().sum().sum())
display(dados_CE['codmun'].isna().sum().sum())

943

487

In [ ]:
dados_CE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84847 entries, 1580 to 195184
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   regiao                  84847 non-null  object 
 1   estado                  84847 non-null  object 
 2   municipio               83904 non-null  object 
 3   coduf                   84847 non-null  int64  
 4   codmun                  84360 non-null  float64
 5   codRegiaoSaude          83904 non-null  float64
 6   nomeRegiaoSaude         83904 non-null  object 
 7   data                    84847 non-null  object 
 8   semanaEpi               84847 non-null  int64  
 9   populacaoTCU2019        84391 non-null  float64
 10  casosAcumulado          84847 non-null  float64
 11  casosNovos              84847 non-null  int64  
 12  obitosAcumulado         84847 non-null  int64  
 13  obitosNovos             84847 non-null  int64  
 14  Recuperadosnovos        0 non-null

aplicando o função excluir dados NaN

In [ ]:
dados_CE = excluiNA(dados_CE, 'municipio','codmun')
dados_CE

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
118343,Nordeste,CE,Abaiara,23,230010.0,23019.0,19ª REGIAO BREJO SANTO,2020-03-27,13,11737.0,0.0,0,0,0,NaN,NaN,0.0
118344,Nordeste,CE,Abaiara,23,230010.0,23019.0,19ª REGIAO BREJO SANTO,2020-03-28,13,11737.0,0.0,0,0,0,NaN,NaN,0.0
118345,Nordeste,CE,Abaiara,23,230010.0,23019.0,19ª REGIAO BREJO SANTO,2020-03-29,14,11737.0,0.0,0,0,0,NaN,NaN,0.0
118346,Nordeste,CE,Abaiara,23,230010.0,23019.0,19ª REGIAO BREJO SANTO,2020-03-30,14,11737.0,0.0,0,0,0,NaN,NaN,0.0
118347,Nordeste,CE,Abaiara,23,230010.0,23019.0,19ª REGIAO BREJO SANTO,2020-03-31,14,11737.0,0.0,0,0,0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195180,Nordeste,CE,Viçosa do Ceará,23,231410.0,23013.0,13ª REGIAO TIANGUA,2021-06-21,25,60889.0,4791.0,3,140,0,NaN,NaN,0.0
195181,Nordeste,CE,Viçosa do Ceará,23,231410.0,23013.0,13ª REGIAO TIANGUA,2021-06-22,25,60889.0,4802.0,11,140,0,NaN,NaN,0.0
195182,Nordeste,CE,Viçosa do Ceará,23,231410.0,23013.0,13ª REGIAO TIANGUA,2021-06-23,25,60889.0,4809.0,7,140,0,NaN,NaN,0.0
195183,Nordeste,CE,Viçosa do Ceará,23,231410.0,23013.0,13ª REGIAO TIANGUA,2021-06-24,25,60889.0,4820.0,11,140,0,NaN,NaN,0.0


Filtrando as informações de colunas que não são necessárias

In [ ]:
dados_CE.columns

Index(['regiao', 'estado', 'municipio', 'coduf', 'codmun', 'codRegiaoSaude',
       'nomeRegiaoSaude', 'data', 'semanaEpi', 'populacaoTCU2019',
       'casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos',
       'Recuperadosnovos', 'emAcompanhamentoNovos', 'interior/metropolitana'],
      dtype='object')

Transformando o `codmun` em **int**

In [ ]:
dados_CE['codmun'] = dados_CE['codmun'].astype(int)

In [ ]:
dados_CE['codmun']

118343    230010
118344    230010
118345    230010
118346    230010
118347    230010
           ...  
195180    231410
195181    231410
195182    231410
195183    231410
195184    231410
Name: codmun, Length: 83904, dtype: int64

Excluir as colunas desnecessárias

In [ ]:
dados_CE =dados_CE.drop(columns = ['regiao','estado','codRegiaoSaude',
       'nomeRegiaoSaude','semanaEpi', 'emAcompanhamentoNovos', 'interior/metropolitana','Recuperadosnovos'])
dados_CE 

,municipio,coduf,codmun,data,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos
118343,Abaiara,23,230010,2020-03-27,11737.0,0.0,0,0,0
118344,Abaiara,23,230010,2020-03-28,11737.0,0.0,0,0,0
118345,Abaiara,23,230010,2020-03-29,11737.0,0.0,0,0,0
118346,Abaiara,23,230010,2020-03-30,11737.0,0.0,0,0,0
118347,Abaiara,23,230010,2020-03-31,11737.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...
195180,Viçosa do Ceará,23,231410,2021-06-21,60889.0,4791.0,3,140,0
195181,Viçosa do Ceará,23,231410,2021-06-22,60889.0,4802.0,11,140,0
195182,Viçosa do Ceará,23,231410,2021-06-23,60889.0,4809.0,7,140,0
195183,Viçosa do Ceará,23,231410,2021-06-24,60889.0,4820.0,11,140,0


Transformando as informações de data em datahora

In [ ]:
dados_CE.dtypes

municipio            object
coduf                 int64
codmun                int64
data                 object
populacaoTCU2019    float64
casosAcumulado      float64
casosNovos            int64
obitosAcumulado       int64
obitosNovos           int64
dtype: object

In [ ]:
dados_CE['data'] = pd.to_datetime(dados_CE['data'],format='%Y-%m-%d')

In [ ]:
dados_CE.dtypes

municipio                   object
coduf                        int64
codmun                       int64
data                datetime64[ns]
populacaoTCU2019           float64
casosAcumulado             float64
casosNovos                   int64
obitosAcumulado              int64
obitosNovos                  int64
dtype: object

Criando a coluna **regiao** 

In [ ]:
dados_CE['regiao']=dados_CE['municipio'].apply(regiao)

Criando a coluna de **ano**

In [ ]:
dados_CE['ano']=pd.DatetimeIndex(dados_CE['data']).year

## DATAFRAMES DOS ANOS - 2021 & 2021

Dataframe relativos aos casos de COVID no Ceará, divididos em anos 2020 e 2021(até o dia 25/06/2021)

In [ ]:
dados_2020=dados_CE.query('ano=="2020"')
dados_2021=dados_CE.query('ano=="2021"')

## DATAFRAMES DAS REGIÕES (ANO 2020 e 2021)

Dataframe relativos aos casos de COVID no Ceará por regiões, divididos em anos 2020 e 2021(até o dia 25/06/2021)

In [ ]:
dados_regiao_2020=dados_2020.groupby(by=['regiao']).sum()
dados_regiao_2020=dados_regiao_2020.drop(columns=['ano','coduf','codmun'],axis=1)
dados_regiao_2020

,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos
regiao,,,,,
Cariri,2.873388e+08,5824562.0,45310,117261,847
Centro-Sul,1.104547e+08,1804621.0,14917,44431,344
Litoral Leste,5.769848e+07,838473.0,6159,24333,141
Litoral Norte,1.128859e+08,2749938.0,15651,64170,347
Litoral Oeste,1.121571e+08,1722985.0,10656,71628,379
Maciço de Baturité,6.885060e+07,1397476.0,8870,36854,204
Região Metropolitana,1.149749e+09,18110356.0,126034,1111890,5616
Serra da Ibiapaba,1.015846e+08,2053856.0,14707,45709,290
Sertao dos Crateús,9.876300e+07,1897705.0,16080,37117,259


In [ ]:
dados_regiao_2021=dados_2021.groupby(by=['regiao']).sum()
dados_regiao_2021=dados_regiao_2021.drop(columns=['ano','coduf','codmun'],axis=1)
dados_regiao_2021

,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos
regiao,,,,,
Cariri,180612960.0,11310309.0,52497,210197,1039
Centro-Sul,69428656.0,4047902.0,22908,85988,402
Litoral Leste,36267616.0,2151036.0,16010,42174,255
Litoral Norte,70956864.0,4201700.0,20065,85708,375
Litoral Oeste,70498736.0,3205405.0,19460,95076,421
Maciço de Baturité,43277520.0,2887543.0,20537,50675,235
Região Metropolitana,722699120.0,42621412.0,242739,1490353,6838
Serra da Ibiapaba,63853152.0,3659335.0,16194,70920,306
Sertao dos Crateús,62079600.0,4220039.0,22369,69552,394


## DATAFRAME DOS MUNICIPIOS GERAL

Dataframe relativos aos casos de COVID no Ceará por municipio

In [ ]:
dados_CE

,municipio,coduf,codmun,data,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,regiao,ano
118343,Abaiara,23,230010,2020-03-27,11737.0,0.0,0,0,0,Cariri,2020
118344,Abaiara,23,230010,2020-03-28,11737.0,0.0,0,0,0,Cariri,2020
118345,Abaiara,23,230010,2020-03-29,11737.0,0.0,0,0,0,Cariri,2020
118346,Abaiara,23,230010,2020-03-30,11737.0,0.0,0,0,0,Cariri,2020
118347,Abaiara,23,230010,2020-03-31,11737.0,0.0,0,0,0,Cariri,2020
...,...,...,...,...,...,...,...,...,...,...,...
195180,Viçosa do Ceará,23,231410,2021-06-21,60889.0,4791.0,3,140,0,Serra da Ibiapaba,2021
195181,Viçosa do Ceará,23,231410,2021-06-22,60889.0,4802.0,11,140,0,Serra da Ibiapaba,2021
195182,Viçosa do Ceará,23,231410,2021-06-23,60889.0,4809.0,7,140,0,Serra da Ibiapaba,2021
195183,Viçosa do Ceará,23,231410,2021-06-24,60889.0,4820.0,11,140,0,Serra da Ibiapaba,2021


In [ ]:
dados_municipios = dados_CE.groupby(by=['municipio','codmun']).sum()
dados_municipios = dados_municipios.drop(columns=['coduf','ano','populacaoTCU2019'],axis=1)
dados_municipios = dados_municipios.reset_index()
dados_municipios=dados_municipios.set_index('codmun')
dados_municipios['regiao'] = dados_municipios['municipio'].apply(regiao)
dados_municipios

,municipio,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,regiao
codmun,,,,,,
230010,Abaiara,72881.0,577,1794,15,Cariri
230015,Acarape,699618.0,2769,6284,27,Maciço de Baturité
230020,Acaraú,1173688.0,4874,24650,103,Litoral Norte
230030,Acopiara,559668.0,3596,22859,140,Centro-Sul
230040,Aiuaba,62230.0,744,1225,11,Sertão dos Inhamus
...,...,...,...,...,...,...
231380,Uruburetama,382540.0,1824,10877,56,Litoral Oeste
231390,Uruoca,274708.0,1451,6973,31,Litoral Norte
231395,Varjota,454691.0,2337,6237,29,Sertão de Sobral


## DATAFRAME DOS MUNICIPIOS/ANO

Dataframe relativos aos casos de COVID no Ceará por municipios, divididos nos anos de 2020 e 2021(até o dia 25/06/2021)

In [ ]:
dados_municipios_2020 = dados_2020.groupby(by=['municipio','codmun']).sum()
dados_municipios_2020 = dados_municipios_2020.drop(columns=['coduf','ano'],axis=1)
dados_municipios_2020 = dados_municipios_2020.reset_index()
dados_municipios_2020=dados_municipios_2020.set_index('codmun')
dados_municipios_2020['regiao'] = dados_municipios_2020['municipio'].apply(regiao)
dados_municipios_2020

,municipio,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,regiao
codmun,,,,,,,
230010,Abaiara,3286360.0,15909.0,160,468,4,Cariri
230015,Acarape,4180120.0,299547.0,1949,3019,14,Maciço de Baturité
230020,Acaraú,17539480.0,513339.0,2760,11285,58,Litoral Norte
230030,Acopiara,15195600.0,173778.0,1366,7238,57,Centro-Sul
230040,Aiuaba,4871720.0,7907.0,76,193,1,Sertão dos Inhamus
...,...,...,...,...,...,...,...
231380,Uruburetama,6118000.0,149762.0,816,4070,22,Litoral Oeste
231390,Uruoca,3875200.0,105279.0,659,2940,17,Litoral Norte
231395,Varjota,5157600.0,169229.0,1235,2539,18,Sertão de Sobral


In [ ]:
dados_municipios_2021 = dados_2021.groupby(by=['municipio','codmun']).sum()
dados_municipios_2021 = dados_municipios_2021.drop(columns=['coduf','ano'],axis=1)
dados_municipios_2021 = dados_municipios_2021.reset_index()
dados_municipios_2021 = dados_municipios_2021.set_index('codmun')
dados_municipios_2021['regiao'] = dados_municipios_2021['municipio'].apply(regiao)
dados_municipios_2021

,municipio,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,regiao
codmun,,,,,,,
230010,Abaiara,2065712.0,56972.0,417,1326,11,Cariri
230015,Acarape,2627504.0,400071.0,820,3265,13,Maciço de Baturité
230020,Acaraú,11024816.0,660349.0,2114,13365,45,Litoral Norte
230030,Acopiara,9551520.0,385890.0,2230,15621,83,Centro-Sul
230040,Aiuaba,3062224.0,54323.0,668,1032,10,Sertão dos Inhamus
...,...,...,...,...,...,...,...
231380,Uruburetama,3845600.0,232778.0,1008,6807,34,Litoral Oeste
231390,Uruoca,2435840.0,169429.0,792,4033,14,Litoral Norte
231395,Varjota,3241920.0,285462.0,1102,3698,11,Sertão de Sobral


## DATAFRAME MAPA

Arquivos geodataframe, contendo a espacialização das informações em arquivos shapefiles 

Incorporando o shape pelo codmun

In [ ]:
df_mapa = gpd.read_file('/content/drive/MyDrive/BOOTCAMP - ALURA/Projeto/Arquivos Projeto 3/shapes/23MUE250GC_SIR_SIRGAS2000.shp')
df_mapa

,ID,CD_GEOCODM,NM_MUNICIP,geometry
0,218,2300101,ABAIARA,"POLYGON ((490898.430 9193553.840, 490786.024 9..."
1,219,2300150,ACARAPE,"POLYGON ((536322.319 9527585.747, 536322.319 9..."
2,220,2300200,ACARAÚ,"POLYGON ((389866.560 9657526.188, 389861.809 9..."
3,221,2300309,ACOPIARA,"POLYGON ((476327.601 9337702.456, 476451.750 9..."
4,222,2300408,AIUABA,"POLYGON ((372974.464 9296802.433, 373011.961 9..."
...,...,...,...,...
179,397,2313807,URUBURETAMA,"POLYGON ((438818.147 9603999.620, 440919.566 9..."
180,398,2313906,URUOCA,"POLYGON ((310312.845 9641811.586, 310751.110 9..."
181,399,2313955,VARJOTA,"POLYGON ((327126.716 9534587.500, 327000.350 9..."
182,400,2314003,VÁRZEA ALEGRE,"POLYGON ((467947.418 9264066.859, 467947.419 9..."


Aplicando  a função **prepara_mapa**

In [ ]:
df_mapa = prepara_mapa(df_mapa,'CD_GEOCODM')
df_mapa=df_mapa.set_index('codmun')

Transformando os mapas em Geo Dataframe

In [ ]:
#Criando os dataframes dos mapas e transformando-os em GeodataFrames
mapa_2020 = pd.merge(dados_municipios_2020, df_mapa, how = 'inner',on = 'codmun')
mapa_2020 = GeoDataFrame(mapa_2020)

#Criando os dataframes dos mapas e transformando-os em GeodataFrames
mapa_2021 = pd.merge(dados_municipios_2021, df_mapa, how = 'inner',on = 'codmun')
mapa_2021 = GeoDataFrame(mapa_2021)

mapa_total = pd.merge(dados_municipios,df_mapa, how = 'inner',on = 'codmun')
mapa_total = GeoDataFrame(mapa_total)

In [ ]:
display(mapa_2020.isnull().sum())

display(mapa_2021.isnull().sum())

display(mapa_total.isnull().sum())

municipio           0
populacaoTCU2019    0
casosAcumulado      0
casosNovos          0
obitosAcumulado     0
obitosNovos         0
regiao              0
geometry            0
id                  0
cod_IBGE            0
dtype: int64

municipio           0
populacaoTCU2019    0
casosAcumulado      0
casosNovos          0
obitosAcumulado     0
obitosNovos         0
regiao              0
geometry            0
id                  0
cod_IBGE            0
dtype: int64

municipio          0
casosAcumulado     0
casosNovos         0
obitosAcumulado    0
obitosNovos        0
regiao             0
geometry           0
id                 0
cod_IBGE           0
dtype: int64

Como visto nenhum dado deu 0 portanto deu tudo certo

# **EXPORTANDO TODOS OS ARQUIVOS**

Resetando os indexs

In [ ]:
dados_CE=dados_CE.reset_index()

dados_2020=dados_2020.reset_index()
dados_2021=dados_2021.reset_index()

dados_regiao_2020=dados_regiao_2020.reset_index()
dados_regiao_2021=dados_regiao_2021.reset_index()

dados_municipios_2020=dados_municipios_2020.reset_index()
dados_municipios_2021=dados_municipios_2021.reset_index()

Exportando os **CSV**

In [ ]:
#Exportando os csv
#1 CEARÁ POR MUNICIPIOS
dados_CE.to_csv('Casos_COVID-19_CEARÁ_.csv',index=False)
files.download('Casos_COVID-19_CEARÁ_.csv')

#2 CADA ANO
dados_2020.to_csv('Casos_COVID-19_CEARÁ_2020.csv',index=False)
files.download('Casos_COVID-19_CEARÁ_2020.csv')

dados_2021.to_csv('Casos_COVID-19_CEARÁ_2021.csv',index=False)
files.download('Casos_COVID-19_CEARÁ_2021.csv')

#3 AGRUPADO POR REGIÃO
dados_regiao_2020.to_csv('Casos_COVID-19_REGIÕES_CEARÁ_2020.csv',index=False)
files.download('Casos_COVID-19_REGIÕES_CEARÁ_2020.csv')

dados_regiao_2021.to_csv('Casos_COVID-19_REGIÕES_CEARÁ_2021.csv',index=False)
files.download('Casos_COVID-19_REGIÕES_CEARÁ_2021.csv')

#3 AGRUPADO POR MUNICIPIO

dados_municipios.to_csv('Casos_COVID-19_MUNICIPIOS_CEARÁ.csv',index=False)
files.download('Casos_COVID-19_MUNICIPIOS_CEARÁ.csv')

dados_municipios_2020.to_csv('Casos_COVID-19_MUNICIPIOS_CEARÁ_2020.csv',index=False)
files.download('Casos_COVID-19_MUNICIPIOS_CEARÁ_2020.csv')

dados_municipios_2021.to_csv('Casos_COVID-19_MUNICIPIOS_CEARÁ_2021.csv',index=False)
files.download('Casos_COVID-19_MUNICIPIOS_CEARÁ_2021.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Exportando os **shapefiles**

In [ ]:
mapa_2020.to_file('Casos_COVID-19_CEARÁ_SHAPEFILE_2020.shp', index=False)
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE_2020.shp')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE_2020.cpg')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE_2020.dbf')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE_2020.prj')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE_2020.shx')

mapa_2021.to_file('Casos_COVID-19_CEARÁ_SHAPEFILE_2021.shp', index=False)
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE_2021.shp')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE_2021.cpg')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE_2021.dbf')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE_2021.prj')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE_2021.shx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Para ver as analises desses dataframes **(INSERIR O CLIQUE AQUI)**

In [ ]:
dados_municipios.to_csv('Casos_COVID-19_MUNICIPIOS_CEARÁ.csv',index=False)
files.download('Casos_COVID-19_MUNICIPIOS_CEARÁ.csv')

mapa_total.to_file('Casos_COVID-19_CEARÁ_SHAPEFILE.shp', index=False)
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE.shp')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE.cpg')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE.dbf')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE.prj')
files.download('Casos_COVID-19_CEARÁ_SHAPEFILE.shx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Referencia



[01] - [Reportagem 1](https://diariodonordeste.verdesmares.com.br/regiao/todas-as-regioes-do-ceara-apresentaram-mais-mortes-por-covid-19-em-2021-que-2020-exceto-a-de-sobral-1.3096088) - *Todas as regiões do Ceará apresentaram mais mortes por Covid-19 em 2021 que 2020, exceto a de Sobral*

[02] - [Reportagem 2](https://diariodonordeste.verdesmares.com.br/regiao/oros-implanta-medidas-restritivas-para-conter-covid-19-veda-aulas-presenciais-e-pontos-turisticos-1.3078419) - *Orós implanta medidas restritivas para conter Covid-19, veda aulas presenciais e pontos turísticos*

[03] - [Reportagem 3](https://diariodonordeste.verdesmares.com.br/regiao/ico-tem-100-de-leitos-de-uti-lotados-desde-inicio-de-maio-1.3093287) - *Icó tem 100% de leitos de UTI lotados desde início de maio*

[04] - [Reportagem 4](https://covidreference.com/timeline_pt) - *Linha do Tempo Covid-19 no mundo*

[05] - [Reportagem 5](https://www.otempo.com.br/interessa/primeiro-caso-de-coronavirus-fora-da-asia-e-registrado-nos-eua-1.2287219) - *Primeiro caso de coronavírus fora da Ásia é registrado nos EUA*

[06] - [Reportagem 6](https://g1.globo.com/ciencia-e-saude/noticia/2020/02/15/primeira-morte-ligada-ao-coronavirus-fora-da-asia-e-confirmada-na-franca.ghtml) - *Primeira morte ligada ao coronavírus fora da Ásia é confirmada na França*

[07] - [Reportagem 7](https://g1.globo.com/sp/sao-paulo/noticia/2020/08/26/primeiro-caso-confirmado-de-covid-19-no-brasil-ocorreu-em-sp-e-completa-seis-meses-nesta-quarta.ghtml) - *Primeiro caso confirmado de Covid-19 no Brasil ocorreu em SP e completa seis meses nesta quarta*

[08] - [Reportagem 8](https://www.conjur.com.br/2021-jan-13/bolsonaro-tentou-sabotar-combate-coronavirus-ong) - *Bolsonaro tentou sabotar combate ao coronavírus, diz Human Rights Watch*

[09] - [Reportagem 9](https://www.uol.com.br/vivabem/noticias/agencia-estado/2021/06/22/preco-que-o-governo-pagou-por-vacina-indiana-covaxin-foi-1000-mais-alto.htm) - *Preço que o governo pagou por vacina indiana Covaxin foi 1000% mais alto*

[10] -[Reportagem 10](https://g1.globo.com/ce/ceara/noticia/2020/03/15/tres-primeiros-casos-de-coronavirus-no-ceara-sao-confirmados-pela-secretaria-da-saude.ghtml) - *Três primeiros casos de coronavírus no Ceará são confirmados pela Secretaria da Saúde*

[11] - [Reportagem 11](https://oglobo.globo.com/epoca/sociedade/ele-esta-incentivando-baderna-desabafo-de-enfermeira-apos-bolsonaro-pedir-que-seguidores-invadam-hospitais-24477843) - *'Ele está incentivando a baderna': o desabafo de enfermeira após Bolsonaro pedir que seguidores invadam hospitais*

[12] - [Reportagem 12](https://g1.globo.com/ce/ceara/noticia/2021/05/09/casos-de-covid-19-no-ceara-em-9-de-maio-de-2021.ghtml) - *Ceará registra mais de 716 mil casos de Covid-19 e 18.538 óbitos pela doença*

[13] - [Reportagem 13](https://www.sanarmed.com/linha-do-tempo-do-coronavirus-no-brasil) - *Linha do tempo do Coronavírus no Brasil*

[14] - [Reportagem 14](https://www.cnnbrasil.com.br/saude/2021/06/07/rio-de-janeiro-82-mil-pessoas-nao-retornaram-para-tomar-segunda-dose-de-vacina)- *Rio de Janeiro: 82 mil pessoas não retornaram para tomar segunda dose de vacina*

[15] - [Portal 1](https://covid.saude.gov.br/) - *Portal oficial do Covid*

[16] - [Documento 1](https://www.ipece.ce.gov.br/wp-content/uploads/sites/45/2014/02/TD_111.pdf) - *IPECE - AS REGIÕES DE PLANEJAMENTO DO
ESTADO DO CEARÁ *

[17] - [Outros 1](https://stackoverflow.com/questions/63974040/line2d-object-has-no-property-column)- Transformando um DATAFRAME em GEODATAFRAME